27. Remove Element

https://leetcode.cn/problems/remove-element/

## 核心思想

本题要求在**原地**、使用 $O(1)$ 额外空间的情况下，移除数组中所有等于 `val` 的元素，并返回新数组的长度。

“原地”意味着不能创建新数组；“移除”也并非一定要从内存中删除，而是将所有不等于 `val` 的元素移动到数组的前面，最终数组的前 `k` 个元素就是结果，其中 `k` 是返回的长度。

解决此问题主要有两大类思路：
1.  **暴力移除法**：直接遍历数组，找到 `val` 就将其删除，并移动后续所有元素。
2.  **双指针法**：通过两个指针的协作，一次遍历就完成所有元素的移动。

---

## 方案一：暴力移除法 ($O(n²)$)

这类方法思路直接，但因为涉及元素的重复移动，时间复杂度较高。

### 1a. 直接使用 `del` 删除 (从后往前)

从后往前遍历，找到目标值就用 `del` 关键字删除。从后往前删除元素，效率比从前往后删略高，因为从后往前删可以减少因删除元素而需要移动后续元素的总次数。

```python
from typing import List

class Solution(object):
    def removeElement(self, nums: List[int], val: int) -> int:
        i = len(nums) - 1
        while i >= 0:
            if nums[i] == val:
                del nums[i]
            i -= 1
        # 技巧：del 会改变原数组的长度，所以 len(nums) 可以直接返回删除后数组的长度
        return len(nums)
```
* **分析**: `del` 操作在列表中删除一个元素后，其后的所有元素都需要向前移动一位，这是一个 $O(n)$ 的操作。在最坏情况下，总时间复杂度为 $O(n²)$。

### 1b. 手动移动元素 (从后往前)

不使用 `del`，手动实现元素的向前移动。这种方式可以更清晰地展示 `del` 操作背后的逻辑。

* **`while` 循环实现**

```python
class Solution1(object):
    def removeElement(self, nums: List[int], val: int) -> int:
        size = len(nums)
        i = size - 1
        while i >= 0:
            if nums[i] == val:
                # 将 i 之后的所有元素向前移动一位
                # 一定要注意第二个参数是 size - 1，j 只能取到数组倒数第二个元素的索引
                for j in range(i, size - 1):
                    nums[j] = nums[j + 1]
                size -= 1 # 有效数组长度减一
            i -= 1
        return size
```

* **`for` 循环实现** (与 `while` 循环逻辑完全一样)

```python
class Solution2(object):
    def removeElement(self, nums, val):
        size = len(nums)
        for i in range(size - 1, -1, -1):
            if nums[i] == val:
                for j in range(i, size - 1):
                    nums[j] = nums[j + 1]
                size -= 1
        return size
```
* **分析**: `for j in range(...)` 这个内层循环就是元素移动的过程，其复杂度与 `del` 操作类似，因此总时间复杂度也为 $O(n²)$。

---

## 方案二：双指针法 ($O(n)$)

这是解决此问题的最优解法，通过一次遍历完成，时间复杂度为 $O(n)$。

### 2a. 快慢指针法

此方法可以保证保留下来的元素的**相对顺序不变**。

* **核心思想**:
    * **快指针 `fast`**: 负责遍历整个原始数组，去寻找所有应该被保留下来的新数组的元素（即不等于 `val` 的元素）。快指针每次循环都加一，访问下一个元素。
    * **慢指针 `slow`**: 指向新数组中下一个应该被填充元素的位置。慢指针以及它之前的所有元素，构成了处理后有效的新数组。只有在发现需要保留的元素时，慢指针才移动。

```python
class Solution3(object):
    def removeElement(self, nums: List[int], val: int) -> int:
        slow, fast = 0, 0
        size = len(nums)
        while fast < size:
            if nums[fast] != val:
                nums[slow] = nums[fast]
                slow += 1
            fast += 1
        # 算法结束后，慢指针的值就是新数组的长度
        return slow
```
* **分析**: 赋值操作的次数等于需要保留的元素数量。

### 2b. 头尾双指针法

此方法**会改变**保留下来的元素的相对顺序，但对于需要删除的元素较少的情况，赋值操作次数更少。

* **核心思想**:
    * **左指针 `left`**: 从头开始，寻找值为 `val` 的元素。
    * **右指针 `right`**: 从尾开始，作为有效数组的边界。
    * 当 `left` 指针找到一个 `val` 时，从数组的最右边（`right` 指针的位置）拿来一个元素，直接覆盖掉当前 `nums[left]` 的值，然后右边界向内收缩 (`right -= 1`)。

```python
class Solution4(object):
    def removeElement(self, nums: List[int], val: int) -> int:
        left, right = 0, len(nums) - 1
        while left <= right:
            if nums[left] == val:
                # 用 right 指针的元素覆盖 left 指针的元素
                nums[left] = nums[right]
                # 右边界向内收缩，因为 nums[right] 的元素已被“用掉”
                right -= 1
                # 关键：此时 left 指针不移动！因为从右边换过来的新值
                # 可能是另一个 val，需要下一轮循环再次检查
            else:
                # 当前元素是需要保留的，左指针向右移动
                left += 1
        # 算法的最后，左指针的值就是新数组的长度
        return left
```
* **分析**: 赋值操作的次数等于需要**删除**的元素数量。